# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [865]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile
import numpy as np

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [866]:
# You may have to adjust the path
corpus = open('./Selma/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [867]:
pattern = 'Nils Holgersson'
width = 25

In [868]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [869]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [870]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [871]:
unique_words_original = set(words)
len(unique_words_original)

44256

Lowercased text

In [872]:
unique_lower = set(map(str.lower, words))
len(unique_lower)

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [873]:
nonletter = r'[^.;:?!\p{L}]'
punctuation = r'[.;:?!]'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [874]:
def clean(text):
    return re.sub(nonletter, ' ', text)

In [875]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [876]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa. Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [877]:
sentence_boundaries = r'[.;:?!]+\p{Z}+(\p{Lu})'

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [878]:
sentence_markup = r' </s>\n<s> \1'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [879]:
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [880]:
print(text)

En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden.


Insert markup codes in the beginning and end of the text

In [881]:
text = '<s> ' + text + ' </s>'

In [882]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga  som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång  hon hade ett stort  grovt ansikte med stränga  mörka drag  hennes händer voro hårda och fulla av sprickor  som barnens hår fastnade i  när hon kammade dem  och till humöret var hon dyster och sorgbunden. </s>


Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [883]:
text = re.sub(r'\p{Z}{2,}', ' ', text)
text = re.sub(r'[.;:?!]', '', text)

In [884]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [885]:
def segment_sentences(text):
    text = clean(text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(sentence_boundaries, sentence_markup, text)
    # text = re.sub(r'\p{Z}{2,}', ' ', text)
    text = re.sub(punctuation, '', text)
    text = text.lower()
    text = '<s> ' + text + ' </s>'
    return text

    

In [886]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [887]:
# corpus = segment_sentences(corpus)
corpus = segment_sentences(corpus)

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [888]:
print(corpus[-557:])

s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare  </s>


You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [889]:
words = re.findall(r'[^ \n]+', corpus)
# words = corpus.split()

The five last lines of the corpus should like this:

In [890]:
print(words[-101:])
# print(words[:20])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [891]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [892]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [893]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [894]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

In [895]:
real_nbr_bigrams = len(frequency_bigrams.items())
print('Real number of bigrams: ', real_nbr_bigrams)
possible_nbr_bigrams = len(set(words))**2
print('Possible number of bigrams: ', possible_nbr_bigrams)
possible_nbr_4grams = len(set(words))**4
print('Possible number of 4grams: ', possible_nbr_4grams)

Real number of bigrams:  320122
Possible number of bigrams:  1683542961
Possible number of 4grams:  2834316901532647521


### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [896]:
def unigram_lm(freq, sent_words):
    print("==============================================================================")
    print("wi\t\tC(wi)\t\t#words\t\tP(wi)")
    print("==============================================================================")
    totWords = 0
    for word in freq:
        totWords += freq[word]
        
    probabilities = [];
    p = {}
    
    
    for w in sent_words:
        p[w]=freq[w]/totWords
        print(w,freq[w],totWords,p[w],sep='\t\t')
        probabilities.append(p[w])
    
    print("==============================================================================")
    entropy = (-1/len(probabilities))*sum(math.log(probability, 2) if probability > 0 else 0 for probability in probabilities)
    sentence_probability = np.prod(probabilities)
        
    print("\nProb. unigrams: " + str(sentence_probability) )
    print("Geometric mean prob.: " + str(np.prod(probabilities)**(1.0/len(probabilities))))
    print("Entropy rate: " + str(entropy))
    print("Perplexity: " + str(2**entropy))
    
    return 2**entropy

In [897]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [898]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi		C(wi)		#words		P(wi)
det		21108		1041560		0.020265755213333847
var		12090		1041560		0.011607588617074388
en		13514		1041560		0.01297476861630631
gång		1332		1041560		0.001278850954337724
en		13514		1041560		0.01297476861630631
katt		16		1041560		1.5361573025077767e-05
som		16288		1041560		0.015638081339529167
hette		97		1041560		9.312953646453396e-05
nils		87		1041560		8.352855332386037e-05
</s>		59047		1041560		0.056690925150735434

Prob. unigrams: 5.3651155337425844e-27
Geometric mean prob.: 0.0023602494649885993
Entropy rate: 8.726844932328587
Perplexity: 423.68402782577465


In [899]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [900]:
def bigram_lm(freq, freq_bigrams, sent_words):
    print("==============================================================================")
    print("wi\t\twi+1\t\tCi,i+1\t\tC(i)\t\tP(wi+1|wi)")
    print("==============================================================================")
    nbr_words = 0
    for word in freq:
        nbr_words += freq[word]
        
    probabilities_array = []
    probabilities_unigram = {}
    probabilities_bigram = {}
    
    
    for i in range(len(sent_words) - 1):
        unigram = sent_words[i]
        bigram = (sent_words[i], sent_words[i + 1])

        unigram_occurences = freq[unigram]
        bigram_occurences = freq_bigrams[bigram] if bigram in freq_bigrams else 0

        probabilities_unigram[unigram] = unigram_occurences / nbr_words
        probabilities_bigram[bigram] = bigram_occurences / unigram_occurences

        bigram_probability_string = str(probabilities_bigram[bigram])
        probability = 0
        if probabilities_bigram[bigram] == 0:
            probability = probabilities_unigram[unigram]
            bigram_probability_string += ' Backoff: ' + str(probabilities_unigram[unigram])
        else:
            probability = probabilities_bigram[bigram]

        probabilities_array.append(probability)
        print(unigram, bigram[1], bigram_occurences, unigram_occurences, bigram_probability_string, sep='\t\t')
    
    print("==============================================================================")
    entropy = (-1/len(probabilities_array))*sum(math.log(probability, 2) if probability > 0 else 0 for probability in probabilities_array)
    sentence_probability = np.prod(probabilities_array)
        
    print("\nProb. unigrams: " + str(sentence_probability))
    print("Geometric mean prob.: " + str(np.prod(probabilities_array)**(1.0/len(probabilities_array))))
    print("Entropy rate: " + str(entropy))
    print("Perplexity: " + str(2**entropy))
    
    return 2**entropy

In [901]:
sent_words = '<s> det var en gång en katt som hette nils </s>'.split()
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi		wi+1		Ci,i+1		C(i)		P(wi+1|wi)
<s>		det		5672		59047		0.09605907158704083
det		var		3839		21108		0.1818741709304529
var		en		712		12090		0.058891645988420185
en		gång		706		13514		0.052242119283705785
gång		en		20		1332		0.015015015015015015
en		katt		6		13514		0.0004439840165754033
katt		som		2		16		0.125
som		hette		45		16288		0.002762770137524558
hette		nils		0		97		0.0 Backoff: 9.312953646453396e-05
nils		</s>		2		87		0.022988505747126436

Prob. unigrams: 2.6492927307096436e-19
Geometric mean prob.: 0.013877556200403469
Entropy rate: 6.171102654133042
Perplexity: 72.05879663243063


In [902]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [903]:
def trigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
        
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams

In [904]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [905]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [906]:
from collections import defaultdict

def def_value():
    return None

def find_word(word,cand_nbr):
    l = defaultdict(def_value)
    len_word = len(word)
    
    for i in frequency_bigrams.items():
        if len(i[0][0]) >= len_word:
            if i[0][0][:len_word] == word:
                if l[i[0][0]] == None:
                    l[i[0][0]] = i[1] 
                else:
                    l[i[0][0]] += i[1]
    
    
    l = sorted(l.items(), key=lambda x: (-x[1], x[0]))

    return [i[0] for i in l[:cand_nbr]]

def find_word1(word,cand_nbr):
    l = []
    len_word = len(word)
    
    for i in frequency.items():
        if len(i[0]) >= len_word:
            if i[0][:len_word] == word:
                l.append(i)
                
    
    
    l = sorted(l, key=lambda x: (-x[1], x[0]))

    return l[:cand_nbr]

print(find_word('de',5))
print(find_word1('de',5))
current_word_predictions_1 = find_word('de',5)

['det', 'de', 'den', 'dem', 'detta']
[('det', 21108), ('de', 11942), ('den', 11624), ('dem', 3196), ('detta', 2482)]


In [907]:
cand_nbr = 5

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [908]:
current_text = "Det var en ".lower()
current_text

'det var en '

Tokenize this text and return a list of tokens. Call it `tokens`.

In [909]:
tokens = tokenize(current_text)

In [910]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [911]:

def find_next(words1,cand_nbr):
    l = []
    for i in trigrams(words).items():
        if i[0][0] == words1[-2] and i[0][1] == words1[-1]:
            l.append((i[0][2],i[1]))
    l = sorted(l, key=lambda x: (-x[1], x[0]))
    return [i[0] for i in l[:cand_nbr]]

next_word_predictions = find_next(tokens,5)
print(next_word_predictions)

['stor', 'liten', 'gammal', 'god', 'sådan']


Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [912]:
current_text = "Det var en g".lower()

In [913]:


def find_word_trigram(word,cand_nbr):
    word = word.split()
    
    l = defaultdict(def_value)
    len_word = len(word[-1])
 
    
    for i in frequency_trigrams.items():
        if len(i[0][2]) >= len_word:
            if i[0][2][:len_word] == word[-1] and i[0][1] == word[-2] and i[0][0] == word[-3]:
                if l[i[0][2]] == None:
                    l[i[0][2]] = i[1] 
                else:
                    l[i[0][2]] += i[1]
    
    
    l = sorted(l.items(), key=lambda x: (-x[1], x[0]))

    return [i[0] for i in l[:cand_nbr]]

current_word_predictions_2 = find_word_trigram(current_text,5)
print(current_word_predictions_2)

['gammal', 'god', 'gång', 'ganska', 'glädje']


In [914]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [915]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'dem', 'detta'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [922]:
STIL_ID = ["fr5536sj-s", "fi1231ka-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [917]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

'{"perplexity_unigrams": 423, "perplexity_bigrams": 72, "current_word_predictions_1": ["det", "de", "den", "dem", "detta"], "next_word_predictions": ["stor", "liten", "gammal", "god", "s\\u00e5dan"], "current_word_predictions_2": ["gammal", "god", "g\\u00e5ng", "ganska", "gl\\u00e4dje"]}'

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [918]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [921]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\filip\\Documents\\Högskola\\[EDAN20] - Språkteknologi\\edan20\\notebooks\\2-language_models_solution.ipynb'

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to include the unigram and bigram tables for five sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 24, 2021.